In [1]:
#Instalar dependencias para instalar Google-Chrome
!sudo apt-get update
!sudo apt-get install -y libu2f-udev
!pip install selenium

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,202 kB]
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,345 kB]
Get:14 h

In [2]:
#Instalar Google Chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f
!rm -rf google-chrome-stable_current_amd64.deb

--2023-05-25 14:17:51--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 172.217.193.93, 172.217.193.136, 172.217.193.190, ...
Connecting to dl.google.com (dl.google.com)|172.217.193.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94682364 (90M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>]  90.30M   307MB/s    in 0.3s    

2023-05-25 14:17:51 (307 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [94682364/94682364]

Selecting previously unselected package google-chrome-stable.
(Reading database ... 122649 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (113.0.5672.126-1) ...
Setting up google-chrome-stable (113.0.5672.126-1) ...
update-alternatives: using /usr/bin/google-chrome-stable to provide /usr/bin/x-www-

In [3]:
# Descargar WebDriver
!wget https://chromedriver.storage.googleapis.com/113.0.5672.24/chromedriver_linux64.zip
!unzip chromedriver_linux64
!rm -rf chromedriver_linux64.zip
!rm -rf LICENSE.chromedriver

--2023-05-25 14:18:06--  https://chromedriver.storage.googleapis.com/113.0.5672.24/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 172.217.193.128, 2607:f8b0:400c:c03::80
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|172.217.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7172785 (6.8M) [application/zip]
Saving to: ‘chromedriver_linux64.zip’

chromedriver_linux6 100%[===================>]   6.84M  --.-KB/s    in 0.06s   

2023-05-25 14:18:06 (105 MB/s) - ‘chromedriver_linux64.zip’ saved [7172785/7172785]

Archive:  chromedriver_linux64.zip
  inflating: chromedriver            
  inflating: LICENSE.chromedriver    


In [4]:
## funcion para convertir el html a txt
def guardar_en_txt(texto, nombre_archivo):
    with open(nombre_archivo, 'w') as archivo:
        archivo.write(texto)

##Primer uso de la busqueda secuencial antes de pasarlo a función

In [5]:
"""soup = BeautifulSoup(driver.page_source, 'html.parser')

# Buscar el id del html
dato_velocidad_grafico_viento = soup.find('span', id='numero_visibilidad_grafico_visibilidad')

# Imprimir el contenido del span específico
if dato_velocidad_grafico_viento:
    print(dato_velocidad_grafico_viento.text.strip())
else:
    print('Span no encontrado')


# Buscar el div específico por clase
dato_velocidad_grafico_viento = soup.find('span', class_='velocidad_grafico_viento')

# Imprimir el contenido del div específico
if dato_velocidad_grafico_viento:
    print(dato_velocidad_grafico_viento.text.strip())
else:
    print('Div no encontrado')"""

"soup = BeautifulSoup(driver.page_source, 'html.parser')\n\n# Buscar el id del html\ndato_velocidad_grafico_viento = soup.find('span', id='numero_visibilidad_grafico_visibilidad')\n\n# Imprimir el contenido del span específico\nif dato_velocidad_grafico_viento:\n    print(dato_velocidad_grafico_viento.text.strip())\nelse:\n    print('Span no encontrado')\n\n\n# Buscar el div específico por clase\ndato_velocidad_grafico_viento = soup.find('span', class_='velocidad_grafico_viento')\n\n# Imprimir el contenido del div específico\nif dato_velocidad_grafico_viento:\n    print(dato_velocidad_grafico_viento.text.strip())\nelse:\n    print('Div no encontrado')"

In [6]:
## Función para buscar tags en html
def buscar_elemento(soup, tag, identificador):
    elemento = soup.find(tag, id=identificador) or soup.find(tag, class_=identificador)
    if elemento:
        return elemento.text.strip()
    else:
        return f'{tag.capitalize()} no encontrado'

In [7]:
## Función para buscar color en Descenso /Estable/ En Ascenso
def buscar_elemento_v(soup, tag, identificador):
    elemento = soup.find(tag, id=identificador) or soup.find(tag, class_=identificador)
    if elemento:
        return elemento
    else:
        return None

In [8]:
## Retorno del valor activo Descenso /Estable/ En Ascenso
def encontrar_activa_v2(x, y, z):
    variables = {'ascenso_color': ascenso_color, 'estable_color': estable_color, 'descenso_color': descenso_color}
    activas = [variable for variable, valor in variables.items() if valor != '#ffffff']
    if len(activas) == 1:
        return activas[0]
    else:
        return "Ninguna o múltiples variables activas"

In [9]:
## Retorno de estado_color hacia estado
def obtener_estado(Presion_Actividad_Pesca, ascenso_color, estable_color, descenso_color):
    if Presion_Actividad_Pesca == 'ascenso_color':
        resultado = 'en ascenso'
    elif Presion_Actividad_Pesca == 'estable_color':
        resultado = 'estable'
    elif Presion_Actividad_Pesca == 'descenso_color':
        resultado = 'en descenso'
    else:
        resultado = "Presión de actividad de pesca desconocida"

    return resultado

In [10]:
## Retorno mensaje de pesca
def obtener_mensaje(Presion_Actividad_Pesca, ascenso_color, estable_color, descenso_color):
    Mensaje = [
        'Muy buena señal. Actividad alta prevista',
        'Actividad normal de los peces',
        'Buena pesca al principio, pero pronto dejarán de alimentarse'
    ]

    if Presion_Actividad_Pesca == 'ascenso_color':
        resultado = Mensaje[0]
    elif Presion_Actividad_Pesca == 'estable_color':
        resultado = Mensaje[1]
    elif Presion_Actividad_Pesca == 'descenso_color':
        resultado = Mensaje[2]
    else:
        resultado = "Presión de actividad de pesca desconocida"

    return resultado

In [11]:
## Estado para la pesca
def obtener_estado_activo_pesca(soup):
    elementos = soup.find_all('div', class_='circulo_estado_grafico_barometro1')
    
    for elemento in elementos:
        if 'circulo_estado_grafico_barometro1_activo' in elemento['class']:
            return elemento.next_sibling.strip()
    
    return "Estado pesca actual no encontrado"

In [189]:
def tabla_zona_costa(soup, id_fila):
    contenedor_principal = soup.find('div', id='tabla_datos_tiempo_datos')
    fila_datos = contenedor_principal.find('div', id=f'tabla_datos_tiempo_datos_{id_fila}')
    tiempo = fila_datos.find('div', class_='tabla_datos_tiempo_datos_tiempo').text
    datos_tiempo = fila_datos.find('div', class_='tabla_datos_tiempo_datos_datos')
    cuadros = datos_tiempo.find_all('div', class_='tabla_datos_tiempo_datos_datos_cuadro')
    resultados = []

    humedad_encontrada = False  # Variable para verificar si se ha encontrado una entrada de humedad válida

    for cuadro in cuadros:
        titulo_element = cuadro.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_titulo')
        numero_element = cuadro.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num')
        unidades_element = cuadro.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_num_unidades')

        titulo = titulo_element.text if titulo_element else ''
        numero = numero_element.text if numero_element else ''
        unidades = unidades_element.text if unidades_element else ''

        # Omitir los resultados con título vacío
        if not titulo:
            continue

        # Solución para capturar el valor de temperatura correctamente
        if titulo == 'Temp.':
            numero_element = cuadro.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_num_grande_color')
            numero = numero_element.text if numero_element else ''

        # Solución para capturar el valor de viento correctamente
        if titulo == 'Viento':
            viento_valor_element = cuadro.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_flecha_numero')
            viento_valor = viento_valor_element.text if viento_valor_element else ''
            resultado = {
                'titulo': titulo,
                'numero': viento_valor,
                'unidades': ''
            }
            resultados.append(resultado)
            continue

        resultado = {
            'titulo': titulo,
            'numero': numero,
            'unidades': unidades
        }
        
        if titulo == 'Humedad' and not humedad_encontrada:
            humedad_numero_element = cuadro.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_grande')
            humedad_numero = humedad_numero_element.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_num_numero').text
            humedad_unidades_element = cuadro.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_num_unidades')
            humedad_unidades = humedad_unidades_element.text if humedad_unidades_element else ''
            if humedad_numero and humedad_unidades:
                resultado_humedad = {
                    'titulo': titulo,
                    'numero': humedad_numero,
                    'unidades': humedad_unidades
                }
                resultados.append(resultado_humedad)
                humedad_encontrada = True  # Marcar la humedad como encontrada
        else:
            resultados.append(resultado)

    presion_influencia_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_presion_influencia')
    if presion_influencia_element is not None:
        presion_influencia = presion_influencia_element.text.strip()
        resultados.append({'Presión Influencia': presion_influencia})

    presion_estado_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_presion_estado')
    if presion_estado_element is not None:
        presion_estado = presion_estado_element.text.strip()
        resultados.append({'Presión Estado': presion_estado})

    sensacion_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_subtitulo')
    if sensacion_element is not None:
        sensacion_valor_element = sensacion_element.find_next_sibling('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_chico')
        if sensacion_valor_element is not None:
            sensacion_valor = sensacion_valor_element.text.strip()
            resultados.append({'Sensación': sensacion_valor})

    punto_rocio_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_subtitulo', string='punto rocío')
    if punto_rocio_element is not None:
        punto_rocio_valor_element = punto_rocio_element.find_next_sibling('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_chico')
        if punto_rocio_valor_element is not None:
            punto_rocio_valor = punto_rocio_valor_element.text.strip()
            resultados.append({'Punto Rocío': punto_rocio_valor})

    viento_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_grande', title='Velocidad del viento (km/h)\ny procedencia del viento')
    if viento_element is not None:
        viento_valor_element = viento_element.find_next_sibling('div', class_='tabla_datos_tiempo_datos_datos_cuadro_flecha_numero')
        if viento_valor_element is not None:
            viento_valor = viento_valor_element.text.strip()
            resultados.append({'Viento': viento_valor})

    direccion_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_direccion')
    if direccion_element is not None:
        direccion_texto_element = direccion_element.find_previous_sibling('div', class_='tabla_datos_tiempo_datos_datos_cuadro_flecha_numero')
        if direccion_texto_element is not None:
            direccion_texto = direccion_texto_element.text.strip()
            resultados.append({'Dirección': direccion_texto})

    resultados = [r for r in resultados if r.get('titulo') != 'Humidity']

    return resultados

In [199]:
def filas_de_zona_costa(soup):
    for i in range(24):
        x = tabla_zona_costa(soup, i)
        print('fila',i,x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7])
        print('\n')

In [186]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# Opciones de Chrome
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')

# Ruta del controlador de Chrome
chrome_driver_path = '/content/chromedriver'

# Crear un objeto Service para especificar la ruta del controlador
service = Service(chrome_driver_path)

# Crear una instancia del navegador Chrome
driver = webdriver.Chrome(service=service, options=chrome_options)

# Navegar a una página web
driver.get('https://tablademareas.com/cl/los-lagos/puerto-montt')
#driver.get('https://tablademareas.com/cl/magallanes/punta-arenas')

# Imprimir HTML to txt
#print(driver.page_source)

# Guardar html en txt
#guardar_en_txt(driver.page_source, 'pagina_entera.txt')

#print(type(driver.page_source))


# Crear un objeto BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

#Viento
Viento = buscar_elemento(soup, 'span', 'velocidad_grafico_viento')
Viento_unidades = buscar_elemento(soup, 'span', 'unidades_grafico_viento')
Rachas = buscar_elemento(soup, 'span', 'rafagas_datos_grafico_tiempo_valor')
print('Viento:',Viento)
print('Unidades de viento:',Viento_unidades,)
print('Procedencia_Viento:',Rachas)
print('\n')

#Section Temperatura
Temperatura = buscar_elemento(soup, 'span', 'temperatura_grafico_termometro')
Temperatura_Unidad = '°C'
Temperatura_Max = buscar_elemento(soup, 'span', 'temperatura_grafico_termometro_max')
Temperatura_Unidad_Max = '°C'
Temperatura_Min = buscar_elemento(soup, 'span', 'temperatura_grafico_termometro_min')
Temperatura_Unidad_Min = '°C'
Temperatura_Sensacion = buscar_elemento(soup, 'span', 'temperatura_grafico_termometro_sensacion')
Temperatura_Unidad_Sensacion = '°C'

print('Temperatura:',Temperatura)
print('Unidad de Temperatura:',Temperatura_Unidad)
print('Temperatura Max:',Temperatura_Max)
print('Unidad de Temperatura Max:',Temperatura_Unidad_Max)
print('Temperatura Min:',Temperatura_Min)
print('Unidad de Temperatura Min:',Temperatura_Unidad_Min)
print('Temperatura Sensacion:',Temperatura_Sensacion)
print('Unidad de Temperatura Sensacion:',Temperatura_Unidad_Sensacion)


print('\n')

# Humedad
Humedad = buscar_elemento(soup, 'span', 'humedad_grafico_humedad')
Unidad_humedad = '%'
Pto_Rocio = buscar_elemento(soup, 'span', 'rocio_datos_grafico_tiempo_valor')
Unidad_Pto_Rocio = '°C'

print('Humedad:',Humedad)
print('Unidad Humedad:',Unidad_humedad)
print('Punto de Rocio:',Pto_Rocio)
print('Unidad Punto de Rocio:',Unidad_Pto_Rocio)
print('\n')

#Tiempo
Visibilidad = buscar_elemento(soup, 'span', 'numero_visibilidad_grafico_visibilidad')
Visibilidad_Unidad = buscar_elemento(soup, 'span', 'unidades_grafico_viento')

print('Visibilidad:',Visibilidad)
print('Visibilidad Unidad:',Visibilidad_Unidad)
print('\n')

#Presion
Presion = buscar_elemento(soup, 'span', 'numeros_presion_grafico_barometro')
Unidad_Presion = buscar_elemento(soup, 'span', 'unidades_presion_grafico_barometro')

print('Presion:',Presion)
print('Unidad Presion:',Unidad_Presion)
print('\n')

## Presión en la actividad pesquera

# Obtener el color de fondo
ascenso_elemento = buscar_elemento_v(soup, 'div', 'leyenda_influencia_grafico_barometro_ascenso')
estable_elemento = buscar_elemento_v(soup, 'div', 'leyenda_influencia_grafico_barometro_estable')
descenso_elemento = buscar_elemento_v(soup, 'div', 'leyenda_influencia_grafico_barometro_descenso')

# Obtener el color de fondo
ascenso_color = ascenso_elemento['style'].split(':')[1].strip()
estable_color = estable_elemento['style'].split(':')[1].strip()
descenso_color = descenso_elemento['style'].split(':')[1].strip()

Color_Presion_Actividad_Pesca=encontrar_activa_v2(ascenso_color,estable_color,descenso_color)     
Presion_Actividad_Pesca=obtener_estado(Color_Presion_Actividad_Pesca, ascenso_color, estable_color, descenso_color)
Mensaje_Presion_Actividad_Pesca = obtener_mensaje(Color_Presion_Actividad_Pesca, ascenso_color, estable_color, descenso_color)
Estado_activo_pesca=obtener_estado_activo_pesca(soup)

print('Color Actividad Pesca:',Color_Presion_Actividad_Pesca)
print('Presion Actividad Pesca:',Presion_Actividad_Pesca)
print('Mensaje Presion Actividad Pesca:',Mensaje_Presion_Actividad_Pesca)
print('Estado activo pesca:',Estado_activo_pesca)

print('\n')

# Cerrar el navegador
driver.quit()


Viento: 11
Unidades de viento: km/h
Procedencia_Viento: 16 km/h


Temperatura: 7
Unidad de Temperatura: °C
Temperatura Max: 9
Unidad de Temperatura Max: °C
Temperatura Min: 6
Unidad de Temperatura Min: °C
Temperatura Sensacion: 5
Unidad de Temperatura Sensacion: °C


Humedad: 87
Unidad Humedad: %
Punto de Rocio: 5
Unidad Punto de Rocio: °C


Visibilidad: 15
Visibilidad Unidad: km/h


Presion: 1009
Unidad Presion: hPa


Color Actividad Pesca: ascenso_color
Presion Actividad Pesca: en ascenso
Mensaje Presion Actividad Pesca: Muy buena señal. Actividad alta prevista
Estado activo pesca: MUY BUENO




In [200]:
filas_de_zona_costa(soup)

fila 0 {'titulo': 'Presión', 'numero': '1012', 'unidades': ''} {'titulo': 'Temp.', 'numero': '7°', 'unidades': ' °C'} {'titulo': 'Humedad', 'numero': '90', 'unidades': ' %'} {'titulo': 'Viento', 'numero': '24', 'unidades': ''} {'Presión Influencia': 'en descenso'} {'Presión Estado': 'Muy bueno'} {'Sensación': '3 °C'} {'Punto Rocío': '6 °C'}


fila 1 {'titulo': 'Presión', 'numero': '1012', 'unidades': ''} {'titulo': 'Temp.', 'numero': '7°', 'unidades': ' °C'} {'titulo': 'Humedad', 'numero': '90', 'unidades': ' %'} {'titulo': 'Viento', 'numero': '24', 'unidades': ''} {'Presión Influencia': 'en descenso'} {'Presión Estado': 'Muy bueno'} {'Sensación': '3 °C'} {'Punto Rocío': '6 °C'}


fila 2 {'titulo': 'Presión', 'numero': '1011', 'unidades': ''} {'titulo': 'Temp.', 'numero': '7°', 'unidades': ' °C'} {'titulo': 'Humedad', 'numero': '90', 'unidades': ' %'} {'titulo': 'Viento', 'numero': '24', 'unidades': ''} {'Presión Influencia': 'en descenso'} {'Presión Estado': 'Muy bueno'} {'Sensación':